---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
# python standard library
from collections import defaultdict

# from pypi
import networkx
import pandas

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one():
    # according to the notes online, the grade doesn't work with data
    # defined outside of the function
    email_network = pandas.read_table('email_network.txt',
                                      dtype={"#Sender": str, "Recipient": str})    
    pandas.read_table('email_network.txt', dtype={"#Sender": str, "Recipient": str})
    # there's a bug in networkx loading MultiDiGraphs from pandas data-frames
    # so this is a work-around
    graph = networkx.MultiDiGraph()
    tuples = [(sender, recipient, {"time": time})
              for (sender, recipient, time) in email_network.values]
    graph.add_edges_from(tuples)
    return graph

In [3]:
one = answer_one()
assert type(one) is networkx.MultiDiGraph

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [4]:
def answer_two():
    one = answer_one()
    return (one.order(), one.size())

In [5]:
employees, emails = answer_two()
assert employees == 167
assert emails == 82927

Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [6]:
def answer_three():
    """decides connectivity based on emails

    First: Assume communication is not necessarily allowed both ways - 
    based on data, can every employee contact each other?

    Second: Assume any contact means there's two way communication. 
    Can every employee be contacted?

    Returns:
     tuple: (every employee contacted every other employee, every employee contacted once)
    """
    emails = answer_one()
    return networkx.is_strongly_connected(emails), networkx.is_weakly_connected(emails)            

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [7]:
def answer_four():
    one = answer_one()
    return len(max(networkx.weakly_connected_component_subgraphs(one), key=len).nodes())

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [8]:
def answer_five():
    """size of largest strongly connected component

    Returns:
     int: number of nodes in largest strongly connected component
    """
    return len(max(networkx.strongly_connected_components(one), key=len))

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [9]:
def answer_six():
    """the sub-graph of the largest strongly connected component
    
    Returns: 
     networkx.MultiDiGraph: largest strongly connected component
    """
    one = answer_one()
    return max(networkx.algorithms.strongly_connected_component_subgraphs(one),
               key=len)

In [10]:
strongly_connected = answer_six()
assert len(strongly_connected.nodes()) == answer_five()
assert type(strongly_connected) is networkx.MultiDiGraph

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [11]:
def answer_seven():
    """average distance between nodes"""
    strongly_connected = answer_six()
    return networkx.average_shortest_path_length(strongly_connected)

In [12]:
average_distance = answer_seven()
print(average_distance)
assert type(average_distance) is float

1.6461587301587302


### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [13]:
def answer_eight():
    """Largest possible distance between employees"""
    strongly_connected = answer_six()
    return networkx.diameter(strongly_connected)

In [14]:
largest_distance = answer_eight()
print(largest_distance)
assert type(largest_distance) is int

3


### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [15]:
def answer_nine():
    """Nodes with eccentricity equal to diameter"""
    strongly_connected = answer_six()
    return set(networkx.periphery(strongly_connected))

In [16]:
nodes = answer_nine()
print(nodes)
assert type(nodes) is set

{'97', '134', '129'}


### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [17]:
def answer_ten():
    """Nodes with eccentricity equal to the radius"""
    strongly_connected = answer_six()
    return set(networkx.center(strongly_connected))

In [18]:
radius_eccentricity = answer_ten()
print(radius_eccentricity)
assert type(radius_eccentricity) is set

{'38'}


### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [19]:
def answer_eleven():
    """node with most shortest paths to other peripheral nodes"""
    candidates = networkx.periphery(strongly_connected)
    outcomes = defaultdict(int)
    for candidate_1 in candidates:
        for candidate_2 in candidates:
            if (candidate_1 != candidate_2
                and networkx.has_path(strongly_connected,
                                      candidate_1,
                                      candidate_2)):
                outcomes[candidate_1] += len(list(networkx.all_shortest_paths(strongly_connected,
                                                                              candidate_1,
                                                                              candidate_2)))
    node = max(outcomes, key=lambda key: outcomes[key])
    return node, outcomes[node]

In [20]:
name,  number = answer_eleven()
print(name, number)
assert type(name) is str
assert type(number) is int

129 15


### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [21]:
def answer_twelve():
    """Smallest number of nodes to remove node from graph"""
    node, count = answer_eleven()    
    center = networkx.center(strongly_connected)
    remove = [networkx.node_connectivity(strongly_connected, node, other) for other in center]
    return max(remove)

In [22]:
answer = answer_twelve()
assert type(answer) is int

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [23]:
def answer_thirteen():
    """undirected version of strongly connected graph"""
    return answer_six().to_undirected()

In [24]:
graph = answer_thirteen()
assert not graph.is_directed()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [25]:
def answer_fourteen():
    graph = answer_thirteen()
    graph = networkx.Graph(graph)
    return networkx.transitivity(graph), networkx.average_clustering(graph)

In [26]:
transitivity, clustering = answer_fourteen()
assert type(transitivity) is float
assert type(clustering) is float